In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
%config IPCompleter.greedy=True

In [9]:
df = pd.read_csv('raw2.csv', encoding='iso-8859-13')
df.head(1)

,PMID/PMCID,Publisher,Journal title,Article title,COST (£) charged to Wellcome (inc VAT when charged)
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00


In [10]:
#rename columns for accuracy & convenience
df.rename(columns={'PMID/PMCID': 'ID','Journal title': 'Journal', 'Article title':'Title',
                   'COST (£) charged to Wellcome (inc VAT when charged)':'Cost'}, inplace=True)
df.head(1)

,ID,Publisher,Journal,Title,Cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00


In [11]:
#Check columntypes...
df.dtypes

ID           object
Publisher    object
Journal      object
Title        object
Cost         object
dtype: object

In [12]:
#remove £ char, and remove commas, which allow me to eventually convert cost to numeric
#check to ensure it looks right

df['Cost_GBP'] = df['Cost'].str.replace(',','').str.replace('£','').str.replace('$','')
df['Journal'] = df['Journal'].str.capitalize()
df.head()

,ID,Publisher,Journal,Title,Cost,Cost_GBP
0,NaN,CUP,Psychological medicine,Reduced parahippocampal cortical thickness in ...,£0.00,0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04,2381.04
2,23043264 PMC3506128,ACS,J med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56,642.56
3,23438330 PMC3646402,ACS,J med chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64,669.64
4,23438216 PMC3601604,ACS,J org chem,Regioselective opening of myo-inositol orthoes...,£685.88,685.88


In [13]:
#drop the unneeded column
df.drop(columns='Cost', inplace=True)
df.head(1)


,ID,Publisher,Journal,Title,Cost_GBP
0,NaN,CUP,Psychological medicine,Reduced parahippocampal cortical thickness in ...,0.00


In [14]:
#Create the cost column as a true float, then drop the old one
df['costGBP'] = df['Cost_GBP'].astype(float)
df.drop(columns='Cost_GBP', inplace=True)

In [15]:
#Costs of 999999 seem to be input as values where they don't know the real cost. 
#create a df without those rows
dfNo9 = df[df.costGBP<999999]
dfNo9.costGBP.max()

201024.0

In [16]:
#Lancet and The lancet and Lancet neurology are presumably the same, so I'll consolidate them:
#yr13Net = yr13Net.assign(netPrice=yr13Net.NPT4_PUB + yr13Net.NPT4_PRIV)
dfNo9 = dfNo9.assign(Journalfix=dfNo9.Journal.str.replace('The lancet','Lancet').str.replace('Lancet neurology','Lancet'))
dfNo9.head()

,ID,Publisher,Journal,Title,costGBP,Journalfix
0,NaN,CUP,Psychological medicine,Reduced parahippocampal cortical thickness in ...,0.00,Psychological medicine
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04,Biomacromolecules
2,23043264 PMC3506128,ACS,J med chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.56,J med chem
3,23438330 PMC3646402,ACS,J med chem,Orvinols with mixed kappa/mu opioid receptor a...,669.64,J med chem
4,23438216 PMC3601604,ACS,J org chem,Regioselective opening of myo-inositol orthoes...,685.88,J org chem


In [17]:
#Create Group By object to see the top 5 by each aggregator
most = dfNo9.groupby(['Journalfix']).agg(['count', 'mean','median', 'std'])

In [18]:
most.columns=['count','mean','median', 'standard_deviation']

In [19]:
most.sort_values(by='count', ascending=False).head()

,count,mean,median,standard_deviation
Journalfix,,,,
Plos one,183,1982.253333,896.99,14173.065922
Journal of biological chemistry,52,1423.588462,1301.14,411.954360
Neuroimage,29,2215.168276,2326.43,266.653947
Nucleic acids research,23,1169.217391,852.00,454.329372
Plos genetics,22,1643.110909,1712.73,153.366825


In [20]:
most.sort_values(by='mean', ascending=False).head()

,count,mean,median,standard_deviation
Journalfix,,,,
Movement disorders,13,17280.909231,2010.240,55209.140040
Public service review,1,6000.000000,6000.000,NaN
Lancet,6,4759.001667,4677.005,535.317831
Cell host & microbe,1,4226.040000,4226.040,NaN
Curr biol.,1,4151.770000,4151.770,NaN


In [21]:
most.sort_values(by='median', ascending=False).head()

,count,mean,median,standard_deviation
Journalfix,,,,
Public service review,1,6000.000000,6000.000,NaN
Lancet,6,4759.001667,4677.005,535.317831
Cell host & microbe,1,4226.040000,4226.040,NaN
Curr biol.,1,4151.770000,4151.770,NaN
Cell journal,1,4041.050000,4041.050,NaN


In [22]:
most.sort_values(by='standard_deviation', ascending=False).head()

,count,mean,median,standard_deviation
Journalfix,,,,
Movement disorders,13,17280.909231,2010.240,55209.140040
Plos one,183,1982.253333,896.990,14173.065922
Gastroenterology,2,1233.040000,1233.040,1407.085926
Public health,2,1352.615000,1352.615,1369.828470
Endeavour,2,1517.715000,1517.715,1275.613562


In [23]:
mod = pd.DataFrame(dfNo9.groupby('Journalfix')['costGBP'].agg(pd.Series.mode))
mod.head()

,costGBP
Journalfix,
Academy of nutrition and dietetics,2379.54
Acs chemical biology,"[947.07, 1267.76, 1294.59, 1294.78, 2286.73]"
Acs chemical neuroscience,1186.8
Acs nano,"[642.89, 693.39]"
"Acta crystallographica section d, biological crystallography",771.42
